In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')
import imblearn

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('../input/train.csv', low_memory=False)
test_df = pd.read_csv('../input/test.csv',low_memory=False)

In [3]:
train_df.set_index("ID_code",inplace=True)
test_df.set_index("ID_code",inplace=True)
target = train_df['target']
train_data = train_df.drop(['target'],axis=1)
test_data = test_df.copy()
train_data.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,...,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train_0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,2.9252,3.1821,14.0137,0.5745,8.7989,14.5691,5.7487,-7.2393,4.2840,30.7133,10.5350,16.2191,2.5791,2.4716,14.3831,13.4325,-5.1488,-0.4073,4.9306,5.9965,-0.3085,12.9041,-3.8766,16.8911,11.1920,10.5785,0.6764,7.8871,4.6667,3.8743,...,15.4576,5.3133,3.6159,5.0384,6.6760,12.6644,2.7004,-0.6975,9.5981,5.4879,-4.7645,-8.4254,20.8773,3.1531,18.5618,7.7423,-10.1245,13.7241,-3.5189,1.7202,-8.4051,9.0164,3.0657,14.3691,25.8398,5.8764,11.8411,-19.7159,17.5743,0.5857,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,-0.4032,8.0585,14.0239,8.4135,5.4345,13.7003,13.8275,-15.5849,7.8000,28.5708,3.4287,2.7407,8.5524,3.3716,6.9779,13.8910,-11.7684,-2.5586,5.0464,0.5481,-9.2987,7.8755,1.2859,19.3710,11.3702,0.7399,2.7995,5.8434,10.8160,3.6783,...,29.4846,5.8683,3.8208,15.8348,-5.0121,15.1345,3.2003,9.3192,3.8821,5.7999,5.5378,5.0988,22.0330,5.5134,30.2645,10.4968,-7.2352,16.5721,-7.3477,11.0752,-5.5937,9.4878,-14.9100,9.4245,22.5441,-4.8622,7.6543,-15.9319,13.3175,-0.3566,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,-0.3249,-11.2648,14.1929,7.3124,7.5244,14.6472,7.6782,-1.7395,4.7011,20.4775,17.7559,18.1377,1.2145,3.5137,5.6777,13.2177,-7.9940,-2.9029,5.8463,6.1439,-11.1025,12.4858,-2.2871,19.0422,11.0449,4.1087,4.6974,6.9346,10.8917,0.9003,...,13.2070,5.8442,4.7086,5.7141,-1.0410,20.5092,3.2790,-5.5952,7.3176,5.7690,-7.0927,-3.9116,7.2569,-5.8234,25.6820,10.9202,-0.3104,8.8438,-9.7009,2.4013,-4.2935,9.3908,-13.2648,3.1545,23.0866,-5.3000,5.3745,-6.2660,10.1934,-0.8417,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,2.3061,2.8102,13.8463,11.9704,6.4569,14.8372,10.7430,-0.4299,15.9426,13.7257,20.3010,12.5579,6.8202,2.7229,12.1354,13.7367,0.8135,-0.9059,5.9070,2.8407,-15.2398,10.4407,-2.5731,6.1796,10.6093,-5.9158,8.1723,2.8521,9.1738,0.6665,...,31.8833,5.9684,7.2084,3.8899,-11.0882,17.2502,2.5881,-2.7018,0.5641,5.3430,-7.1541,-6.1920,18.2366,11.7134,14.7483,8.1013,11.8771,13.9552,-10.4701,5.6961,-3.7546,8.4117,1.8986,7.2601,-0.4639,-0.0498,7.9336,-12.8279,12.4124,1.8489,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,-9.4458,-12.1419,13.8481,7.8895,7.7894,15.0553,8.4871,-3.0680,6.5263,11.3152,21.4246,18.9608,10.1102,2.7142,14.2080,13.5433,3.1736,-3.3423,5.9015,7.9352,-3.1582,9.4668,-0.0083,19.3239,12.4057,0.6329,2.7922,5.8184,19.3038,1.4450,...,33.5107,5.6953,5.4663,18.2201,6.5769,21.2607,3.2304,-1.7759,3.1283,5.5518,1.4493,-2.6627,19.8056,2.3705,18.4685,16.3309,-3.3456,13.5261,1.7189,5.1743,-7.6938,9.7685,4.8910,12.2198,11.8503,-7.8931,6.4209,5.9270,16.0201,-0.2829,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [4]:
# import imblearn
# from imblearn.over_sampling import RandomOverSampler
# from collections import Counter
# print("Count before oversampling:  ")
# print(Counter(target))
# oversample = RandomOverSampler(sampling_strategy='minority')
# # oversample = RandomOverSampler(sampling_strategy=0.5)

# X_over, y_over = oversample.fit_resample(train_data, target)
# print("count ater oversampling : ")
# print(Counter(y_over))

In [5]:
from sklearn.linear_model import LogisticRegression
import time
from sklearn import metrics
from sklearn.model_selection import KFold

In [6]:
model=LogisticRegression(C=1,n_jobs=4,penalty="l2")

In [7]:
train = train_data.copy()
test = test_data.copy()
kford=KFold(n_splits=5,random_state=2,shuffle=True)
start_time=time.time()
aucs=[]

test_preds=[]
# for early stopping
# it takes a long time if using all the samples.
samples=train.shape[0]//20
for ford,(train_idx,val_idx) in enumerate(kford.split(train[:samples],target[:samples])):
    print("####################################")
    print("############ford:",ford)
    sample_x=train.iloc[train_idx].values
    sample_y=target.iloc[train_idx].values
    
    sample_val_x=train.iloc[val_idx].values
    sample_val_y=target.iloc[val_idx].values
    
    model.fit(sample_x,sample_y)
    y_pred_prob=model.predict_proba(sample_x)[:,1]
    y_val_pred_prob=model.predict_proba(sample_val_x)[:,1]
    
    train_auc=metrics.roc_auc_score(sample_y,y_pred_prob)
    val_auc=metrics.roc_auc_score(sample_val_y,y_val_pred_prob)
    print("train auc:{},val auc:{}".format(train_auc,val_auc))
    aucs.append([train_auc,val_auc])
    test_preds.append(model.predict_proba(test)[:,1])
    
end_time=time.time()
val_aucs=[auc[1] for auc in aucs]
print("using {} samples,total time:{}s,mean val auc:{}".format(samples,end_time-start_time,np.mean(val_aucs)))

####################################
############ford: 0
train auc:0.8671085997172423,val auc:0.8265702400883719
####################################
############ford: 1
train auc:0.868515880774527,val auc:0.8172658838259405
####################################
############ford: 2
train auc:0.8692247463084699,val auc:0.8178386637801326
####################################
############ford: 3
train auc:0.8662467115634815,val auc:0.8296777777777777
####################################
############ford: 4
train auc:0.8674150987041669,val auc:0.8284286507495837
using 10000 samples,total time:51.21026921272278s,mean val auc:0.8239562432443612


In [8]:
test_preds=pd.DataFrame(test_preds).T
test_preds.index=test.index

In [9]:
submission=test_preds.mean(axis=1)
submission=pd.DataFrame(submission)
submission.columns=["target"]
submission.head()
submission.to_csv("submission.csv")